In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('Biochemical_results_BGEM.csv')

In [3]:
raw = pd.read_csv('Raw_spectrum_merged.csv')

In [4]:
data = data.loc[:, ['Sample ID', '% N', '% P', '% K', '% S', '% Ca', '% Mg', 'ppm Zn', 'ppm Fe', 'ppm Mn',
       'ppm Cu', 'ppm B', 'ppm Mo' ]]

In [5]:
new_ID = []

for i in data['Sample ID']:
    str(i)
    ID = i[0:4]
    new_ID.append(ID)

In [6]:
data['Sample ID'] = new_ID

In [7]:
data.rename(columns={'Sample ID' : 'PLOT ID'}, inplace= True)

In [8]:
data['PLOT ID'] =data['PLOT ID'].astype('int')

In [9]:
merged= pd.merge(data, raw, how='outer')
merged.sort_values(by='PLOT ID', inplace=True)
merged.reset_index(inplace=True, drop=True)

In [10]:
len(merged.columns)

2175

In [11]:
merged.columns[0:50]

Index(['PLOT ID', '% N', '% P', '% K', '% S', '% Ca', '% Mg', 'ppm Zn',
       'ppm Fe', 'ppm Mn', 'ppm Cu', 'ppm B', 'ppm Mo', 'rows', 'ranges',
       'Block', 'Rep', 'Group', 'Trt', 'year', 'genotype', 'note',
       'Calibration', 'ASD', '350', '351', '352', '353', '354', '355', '356',
       '357', '358', '359', '360', '361', '362', '363', '364', '365', '366',
       '367', '368', '369', '370', '371', '372', '373', '374', '375'],
      dtype='object')

In [12]:
a=[0,13,14,15,16,17,18,19,20,21,22,23,1,2,3,4,5,6,7,8,9,10,11,12]
b =(np.arange(24,2175).tolist())
merged2 =merged.iloc[:, a +b]

In [13]:
merged2.columns[0:30]

Index(['PLOT ID', 'rows', 'ranges', 'Block', 'Rep', 'Group', 'Trt', 'year',
       'genotype', 'note', 'Calibration', 'ASD', '% N', '% P', '% K', '% S',
       '% Ca', '% Mg', 'ppm Zn', 'ppm Fe', 'ppm Mn', 'ppm Cu', 'ppm B',
       'ppm Mo', '350', '351', '352', '353', '354', '355'],
      dtype='object')

In [167]:
merged.to_csv('Spectrum_with_biochemical_traits.csv', index= False)


# removing outliers

In [80]:
import pandas as pd
import numpy as np

In [81]:
data = pd.read_csv('Spectrum_with_biochemical_traits.csv')

In [82]:
data.columns[0:30]

Index(['PLOT ID', '% N', '% P', '% K', '% S', '% Ca', '% Mg', 'ppm Zn',
       'ppm Fe', 'ppm Mn', 'ppm Cu', 'ppm B', 'ppm Mo', 'rows', 'ranges',
       'Block', 'Rep', 'Group', 'Trt', 'year', 'genotype', 'note',
       'Calibration', 'ASD', '350', '351', '352', '353', '354', '355'],
      dtype='object')

In [83]:
data['Group']

0       Inbred
1       Inbred
2       Inbred
3       Inbred
4       Inbred
         ...  
2395    Hybrid
2396    Hybrid
2397    Hybrid
2398    Hybrid
2399    Hybrid
Name: Group, Length: 2400, dtype: object

In [84]:
traits = ['% N','% P', '% K', '% S', '% Ca', '% Mg', 'ppm Zn', 'ppm Fe', 'ppm Mn', 'ppm Cu', 'ppm B', 'ppm Mo']
groups = ['Inbred', 'Hybrid']
trts = ['HN', 'LN']

In [7]:
#data_non_na = data.loc[data[trait].notna(),:]
#data_non_na = data_non_na.loc[data_non_na[trait].notna(), :]

In [85]:
#defining a function to remove the outliers
def outlier_treatment(datacolumn):
 sorted(datacolumn)
 Q1=datacolumn.quantile(0.25)
 Q3=datacolumn.quantile(0.75)
 #Q1,Q3 = np.percentile(datacolumn , [25,75])
 IQR = Q3 - Q1
 lower_range = Q1 - (1.5 * IQR)
 upper_range = Q3 + (1.5 * IQR)
 return lower_range,upper_range

In [65]:
#Outlier treatment
for group in groups:
    for trt in trts:
        for trait in traits:
            data_non_na = data.loc[data[trait].notna(),:]
            outlier_treatment(data_non_na.loc[(data_non_na['Group'] == group) & (data_non_na['Trt'] == trt) , trait])
            lowerbound,upperbound = outlier_treatment(data_non_na.loc[(data_non_na['Group'] == group) & (data_non_na['Trt'] == trt) , trait])
            rm_index =data_non_na.loc[(data_non_na['Group'] == group) & (data_non_na['Trt'] == trt) , trait][(data_non_na.loc[(data_non_na['Group'] == group) & (data_non_na['Trt'] == trt)  , :][trait] < lowerbound) | (data_non_na.loc[(data_non_na['Group'] == group) & (data_non_na['Trt'] == trt) , :][trait] > upperbound)].index.values.tolist()
            rm_index
            data.loc[rm_index, trait] = np.nan


In [87]:
data.to_csv('Spectrum_with_biochemical_traits_outlier_removal.csv')

In [27]:
data_non_na.loc[data_non_na['Group'] == group & data_non_na['Trt'] == trt , trait]

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]